# SPAM or HAM

In [11]:
train_spam = ['send us your password', 'review our website', 'send your password', 'send us your account']
train_ham = ['Your activity report','benefits physical activity', 'the importance vows']
test_emails = {'spam':['renew your password', 'renew your vows'], 'ham':['benefits of our account', 'the importance of physical activity']}

In [12]:
# make a vocabulary of unique words that occur in known spam emails
vocab_words_spam = []

for sentence in train_spam:
    sentence_as_list = sentence.split()
    for word in sentence_as_list:
        vocab_words_spam.append(word)     
vocab_words_spam = list(set(vocab_words_spam))

print(vocab_words_spam)

['us', 'review', 'password', 'our', 'your', 'website', 'account', 'send']


In [13]:
vocab_words_ham = []

for sentence in train_ham:
    sentence_as_list = sentence.split()
    for word in sentence_as_list:
        vocab_words_ham.append(word)
vocab_words_ham = list(set(vocab_words_ham))

print(vocab_words_ham)

['the', 'Your', 'physical', 'activity', 'benefits', 'vows', 'report', 'importance']


In [14]:
def compute_likelihood(vocab, train):
    likelihood = {}
    for w in vocab:
        count = 0
        for sentence in train:
            if w in sentence:
                #print(w+":", sentence)
                count += 1
        print(f"Number of ham emails with the word '{w}': {count}")
        prob = (count + 1)/(len(train) + 2) # smoothing
        print(f"Hamicity of the word '{w}': {prob} ")
        likelihood[w.lower()] = prob
    return likelihood

In [15]:
likelihood_spam = compute_likelihood(vocab_words_spam, train_spam)
likelihood_spam

Number of ham emails with the word 'us': 2
Hamicity of the word 'us': 0.5 
Number of ham emails with the word 'review': 1
Hamicity of the word 'review': 0.3333333333333333 
Number of ham emails with the word 'password': 2
Hamicity of the word 'password': 0.5 
Number of ham emails with the word 'our': 4
Hamicity of the word 'our': 0.8333333333333334 
Number of ham emails with the word 'your': 3
Hamicity of the word 'your': 0.6666666666666666 
Number of ham emails with the word 'website': 1
Hamicity of the word 'website': 0.3333333333333333 
Number of ham emails with the word 'account': 1
Hamicity of the word 'account': 0.3333333333333333 
Number of ham emails with the word 'send': 3
Hamicity of the word 'send': 0.6666666666666666 


{'us': 0.5,
 'review': 0.3333333333333333,
 'password': 0.5,
 'our': 0.8333333333333334,
 'your': 0.6666666666666666,
 'website': 0.3333333333333333,
 'account': 0.3333333333333333,
 'send': 0.6666666666666666}

In [16]:
likelihood_ham = compute_likelihood(vocab_words_ham, train_ham)
likelihood_ham

Number of ham emails with the word 'the': 1
Hamicity of the word 'the': 0.4 
Number of ham emails with the word 'Your': 1
Hamicity of the word 'Your': 0.4 
Number of ham emails with the word 'physical': 1
Hamicity of the word 'physical': 0.4 
Number of ham emails with the word 'activity': 2
Hamicity of the word 'activity': 0.6 
Number of ham emails with the word 'benefits': 1
Hamicity of the word 'benefits': 0.4 
Number of ham emails with the word 'vows': 1
Hamicity of the word 'vows': 0.4 
Number of ham emails with the word 'report': 1
Hamicity of the word 'report': 0.4 
Number of ham emails with the word 'importance': 1
Hamicity of the word 'importance': 0.4 


{'the': 0.4,
 'your': 0.4,
 'physical': 0.4,
 'activity': 0.6,
 'benefits': 0.4,
 'vows': 0.4,
 'report': 0.4,
 'importance': 0.4}

In [17]:
prior_spam = len(train_spam) / (len(train_spam)+(len(train_ham)))
print(f'Prior prob HAM: {prior_spam}')
prior_ham = len(train_ham) / (len(train_spam)+(len(train_ham)))
print(f'Prior prob HAM: {prior_ham}')

Prior prob HAM: 0.5714285714285714
Prior prob HAM: 0.42857142857142855


In [18]:
from functools import reduce

def Bayes(txt):
    probs_spam = []
    probs_ham = []

    txt_as_list = txt.split()
    for w in txt_as_list:
        if w in likelihood_spam:
            pr_WS = likelihood_spam[w]
        else:
            pr_WS = 1.0/(len(train_spam)+2)
        
        if w in likelihood_ham:
            pr_WH = likelihood_ham[w]
        else:
            pr_WH = 1.0/(len(train_ham)+2)
        
        probs_spam.append(pr_WS)
        probs_ham.append(pr_WH)
    
    p_if_spam = prior_spam * reduce(lambda num1, num2: num1 * num2, probs_spam, 1.0)
    p_if_ham = prior_ham * reduce(lambda num1, num2: num1 * num2, probs_ham, 1.0)
    return p_if_spam / (p_if_spam + p_if_ham)

In [19]:
for sentence in train_spam:
    prob_spam = Bayes(sentence)
    print(f'{sentence} -> {prob_spam}')

send us your password -> 0.9788566953797965
review our website -> 0.9391435011269722
send your password -> 0.9487666034155597
send us your account -> 0.9686168151879117


In [20]:
for sentence in train_ham:
    prob_spam = Bayes(sentence)
    print(f'{sentence} -> {prob_spam}')

Your activity report -> 0.11394712853236098
benefits physical activity -> 0.06041565973900434
the importance vows -> 0.08796622097114706
